# Functional Programming

## FP 101

- Immutability
- Functions as First Class Citizens
- Pure Functions

### Functions as First Class Citizens

We can pass functions as arguments to other functions.



In [14]:
// Install dependencies
#r "nuget: xunit, 2.4.2"
#r "nuget: FluentAssertions, 6.11.0"
#r "nuget: CSharpFunctionalExtensions, 2.39.2"
using Xunit;
using FluentAssertions;
using FluentAssertions.Extensions;
using CSharpFunctionalExtensions;

Installed Packages CSharpFunctionalExtensions, 2.39.2 FluentAssertions, 6.11.0 xunit, 2.4.2

In [15]:
// Higher order function: a function that takes a function as an argument or returns a function as a result
public static string Apply(Func<string, string> changeFcn, string input)
{
    return changeFcn(input);
}

public static string MyToUpper(string input) => input.ToUpper();
public static string MyToLower(string input) => input.ToLower();

var homer = "Homer";

var step1 = Apply(MyToUpper, homer);
step1.Should().Be("HOMER");

var step2 = Apply(MyToLower, step1);
step2.Should().Be("homer");

### Pure Functions

A pure function is a function that has no side effects. It does not modify any external state.

Example:

```csharp
// Pure
public int Add(int a, int b) => a + b;
```

```csharp
// Impure
public int Add(int a, int b)
{
    var result = a + b;
    Console.WriteLine(result);
    return result;
}
```

## Challenge: Combining many small functions to larger functions

But: The output of one function has to "fit" as input to the next function.

### Maybe "wrapper" is more powerful than you think!

Be using C# extension methods we can build a simple mapping function to chain functions.

In [16]:
public static Maybe<TOut> Map<TIn, TOut>(this Maybe<TIn> input, Func<TIn, TOut> map)
{
    if (input.HasValue)
    {
        return map(input.Value);
    }
    else
    {
        return Maybe<TOut>.None;
    }
}

var maybeHomer = Maybe<string>.From(homer);
var maybeHomerUpper = maybeHomer.Map(MyToUpper);
maybeHomerUpper.Should().Be(Maybe<string>.From("HOMER"));

var maybeNone = Maybe<string>.None;
var maybeNoneUpper = maybeNone.Map(MyToUpper);
maybeNoneUpper.Should().Be(Maybe<string>.None);

This is basically how LINQ `Select` works! 

Except that LINQ `Select` only works with `IEnumerable<T>`.

In functional lingo, this is called a `Functor`.

![Functor](./images/Funktor_1.png)

### Functor ("Mappable")

A container that can be mapped over.

```haskell
map: (a -> b) -> F a -> F b
```

Other names: `fmap` (Haskell), `Select` (LINQ), `map` (JavaScript), `<$>`, `<!>` (infix operators)

### Result "wrapper" is even more powerful than you think!

Be using C# extension methods we can build simple bind function to chain functions.

In [17]:
public static Result<K> Bind<T, K>(this Result<T> result, Func<T, Result<K>> func)
{
    if (result.IsFailure)
    {
        return Result.Failure<K>(result.Error);
    }
    else
    {
        return func(result.Value);
    }
}

This is basically how LINQ `SelectMany` works! 

Except that LINQ `SelectMany` only works with `IEnumerable<T>`.

In functional lingo, this is called a `Monad`.

![Monad](./images/Monade_1.png)

### Monad ("Chainable")

A container that has a `Bind` function (following certain rules).

```haskell
bind: (a -> M b) -> M a -> M b
```

Other names: `>>=` (Haskell), `SelectMany` (LINQ), `chain` (JavaScript), `>>=`, `>=>` (infix operators)